# Back Testing

This notebook demonstrates several approaches to running back tests: 

* Single Run
* Walk Forward 
* Monte Carlo Simulation

In [1]:
%use roboquant(3.0.0-SNAPSHOT)
Welcome()

roboquant 3.0.0-SNAPSHOT 
 build: 2025-05-02T09:27:32Z 
 home: /Users/peter/.roboquant 
 os: Mac OS X 15.4.1 
 jvm: OpenJDK 64-Bit Server VM 19.0.2 
 memory: 4096 MB 
 cpu cores: 8

# Single Run
For the back testing examples in this notebook, we'll use a data feed, strategy and metric that come out of the box with _roboquant_.

# Single Run
The most simple back test approach is to run a single back test over all the data available in the feed.


In [2]:
// The data feed we'll be using 
val feed = AvroFeed.sp25()

// Use the default included Exponential Moving Average Crossover strategy
val strategy = EMACrossover()

// For this notebook we'll only monitor some key Account metrics
val journal = MemoryJournal(AccountMetric())

val account = run(feed, strategy, journal=journal)
account

last update  : 2024-12-31T21:00:00Z
cash         : USD 940,894.94
buying Power : USD 940,894.94
equity       : USD 1,134,824.62
positions    : 0@WMT, 0@JPM, 95@XOM, 78@NVDA, 24@BRK.B, 0@MA, 0@META, 63@ABBV, 41@V, 0@CVX, 48@AAPL, 17@NFLX, 54@AMZN, 68@JNJ, 0@HD, 61@AVGO, 19@UNH, 67@GOOGL, 66@GOOG, 0@MSFT, 46@TSLA, 0@PG, 14@LLY, 0@MRK, 12@COST
open orders  : -17@NFLX, -78@NVDA, -54@AMZN

## Inspect the account

Likely the first thing you want to know after the run has finished is: how does the account look like at the end of the run? 

An account in roboquant is owned by the broker and contains the following 4 types of data: 

1. the cash balances
2. the open positions
3. the open orders


In [3]:
// You can also get more details, for example about the assets in the portfolio
for ((asset,position) in account.positions) {
    println("${asset.symbol} => ${position.size}")
}

WMT => 0
JPM => 0
XOM => 95
NVDA => 78
BRK.B => 24
MA => 0
META => 0
ABBV => 63
V => 41
CVX => 0
AAPL => 48
NFLX => 17
AMZN => 54
JNJ => 68
HD => 0
AVGO => 61
UNH => 19
GOOGL => 67
GOOG => 66
MSFT => 0
TSLA => 46
PG => 0
LLY => 14
MRK => 0
COST => 12


## Inspect metrics
The account contains the state at the end of the run. But metrics are more powerful in that they capture state during each step of the run. So they give an overview how a metric evolves over time during the run. It is the logger who is responsible for storing or logging the metric results.  

In [4]:
journal.getMetricNames()

[account.orders, account.positions, account.cash, account.buyingpower, account.equity, account.mdd]

In [5]:
val equity = journal.getMetric("account.equity")
TimeSeriesChart(equity)

<!-- roboquant TimeSeriesChart -->

# Walk Forward

To move from a single run to a walk forward back test is simple and requires only a few lines of extra code. We run the back test multiple times with each iteration limited to a certain timeframe. 

In [6]:
feed.timeframe.split(1.years).forEach { 
    val account = run(feed, EMACrossover(), timeframe = it)
    println("timeframe=$it equity=${account.equity()}")
}

timeframe=[2021-01-04T21:00:00Z - 2022-01-04T21:00:00Z> equity=USD 1,042,048.59
timeframe=[2022-01-04T21:00:00Z - 2023-01-04T21:00:00Z> equity=USD 984,506.12
timeframe=[2023-01-04T21:00:00Z - 2024-01-04T21:00:00Z> equity=USD 1,031,151.48
timeframe=[2024-01-04T21:00:00Z - 2024-12-31T21:00:00Z] equity=USD 1,037,149.01


# Monte Carlo Simulation

The problem with back testing is that we don't for sure which historic period is most representative for the present. So instead of selecting a set of predefined periods, we'll draw random periods (uniform distributed) from our feed. 

In [7]:
// Run 25 back-tests each of 6 months
feed.timeframe.sample(6.months, 25).forEach {
    val account = run(feed, EMACrossover(), timeframe = it)
    println("timeframe=$it equity=${account.equity()}")
}

timeframe=[2021-09-07T00:00:00Z - 2022-03-07T00:00:00Z> equity=USD 1,004,318.41
timeframe=[2021-01-26T00:00:00Z - 2021-07-26T00:00:00Z> equity=USD 1,016,613.42
timeframe=[2023-11-17T00:00:00Z - 2024-05-17T00:00:00Z> equity=USD 1,015,281.86
timeframe=[2021-08-03T00:00:00Z - 2022-02-03T00:00:00Z> equity=USD 1,018,589.38
timeframe=[2021-01-21T00:00:00Z - 2021-07-21T00:00:00Z> equity=USD 1,013,591.36
timeframe=[2021-08-09T00:00:00Z - 2022-02-09T00:00:00Z> equity=USD 1,017,253.81
timeframe=[2024-06-09T00:00:00Z - 2024-12-09T00:00:00Z> equity=USD 1,023,752.01
timeframe=[2023-03-02T00:00:00Z - 2023-09-02T00:00:00Z> equity=USD 1,006,045.08
timeframe=[2022-11-07T00:00:00Z - 2023-05-07T00:00:00Z> equity=USD 1,012,522.58
timeframe=[2021-04-01T00:00:00Z - 2021-10-01T00:00:00Z> equity=USD 1,013,834.61
timeframe=[2024-05-06T00:00:00Z - 2024-11-06T00:00:00Z> equity=USD 1,009,844.90
timeframe=[2021-09-02T00:00:00Z - 2022-03-02T00:00:00Z> equity=USD 1,002,856.83
timeframe=[2021-09-21T00:00:00Z - 2022-0